In [1]:
import sqlite3

# all the verbs, ordered
QUERY = '''
  select a.verb from appdb a
  ORDER BY a.freq DESC
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
verbs = [row[0] for row in c.execute(QUERY)]
len(verbs), verbs[:5], verbs[-5:]

(1974,
 ['andare', 'aprire', 'avere', 'bere', 'cadere'],
 ['bluffare', 'spettegolare', 'fraternizzare', 'pomiciare', 'internalizzare'])

In [2]:
# just the irregular verbs
QUERY = '''
  select a.verb from appdb a
  WHERE NOT a.irregular
  ORDER BY a.freq DESC  
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
regulars = [row for row in c.execute(QUERY)]
regulars[:5], regulars[-5:]

([('farsi',), ('darsi',), ('trovare',), ('trovarsi',), ('vedere',)],
 [('bluffare',),
  ('spettegolare',),
  ('fraternizzare',),
  ('pomiciare',),
  ('internalizzare',)])

In [3]:
# just the irregular verbs
QUERY = '''
  select a.verb from appdb a
  WHERE a.irregular
  ORDER BY a.freq DESC  
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
irregulars = [row[0] for row in c.execute(QUERY)]
irregulars[:5], irregulars[-5:]

(['andare', 'aprire', 'avere', 'bere', 'cadere'],
 ['stare', 'tradurre', 'uscire', 'venire', 'volere'])

In [10]:
import random
from collections import defaultdict

def genderizer(pos):
    if pos == 2: return(random.choice([2, 3]))
    if pos == 6: return(random.choice([6, 7]))
    return(pos)   

def pick_pos(k):
    options = [0,1,2,4,5,6]
    choices = random.sample(options, k=k)
    return([str(genderizer(choice)) for choice in choices])

for i in range(10):
    print(sorted(pick_pos(3)))

['1', '3', '6']
['0', '3', '6']
['1', '4', '7']
['1', '3', '4']
['0', '1', '4']
['0', '1', '2']
['1', '3', '5']
['0', '1', '7']
['0', '4', '6']
['3', '4', '5']


In [ ]:
# query a single verb
QUERY = '''
  select * FROM cards c 
  WHERE c.verb = '%s'
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
forms = [row for row in c.execute(QUERY % 'dare')]
len(forms)

In [ ]:
# A1 deck
QUERY = '''
  select a.*, c.ROWID, c.pos, c.verb, c.form, c.conjugation, f.level from cards c
  INNER JOIN appdb a ON c.verb = a.verb 
  INNER JOIN forms f ON f.form = c.form
  WHERE c.form = 'INDICATIVO_PASSATO_PROSSIMO' AND a.irregular
  ORDER BY a.freq DESC
  LIMIT 1
  '''

QUERY = '''
  select a.*, c.ROWID, c.pos, c.verb, c.form, c.conjugation, f.level from cards c
  INNER JOIN appdb a ON c.verb = a.verb 
  INNER JOIN forms f ON f.form = c.form
  WHERE c.form = 'INDICATIVO_PASSATO_PROSSIMO' AND a.irregular
  ORDER BY a.freq DESC
  '''

conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
results = [row for row in c.execute(QUERY)]
results

In [56]:
x = [x for x in range(20)]
x[2:6]

[2, 3, 4, 5]

In [51]:
QUERY = '''
  select c.ROWID, c.verb, c.pos, c.form, c.conjugation, '{level2}' from cards c 
  WHERE verb = '{verb}' 
  AND c.form ='{form}'
  AND pos IN ({pos})
  '''

def one_form(c, form, k, start, end, level1):
    cards = []
    for verb in verbs[start:end]:
        pos = pick_pos(k)
        q = QUERY.format(verb=verb, form=form, pos=','.join(pos), level2=level1)
        for row in c.execute(q):
            cards += [row]
    return(cards)
    

conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
cards = []
# A1 0 - 50
cards += one_form(c, 'INDICATIVO_PASSATO_PROSSIMO',    3, 0, 50, "A1")
cards += one_form(c, 'INDICATIVO_PASSATO_PROSSIMO',    1, 51, 250, "A2")
cards += one_form(c, 'INDICATIVO_PASSATO_PROSSIMO',    1, 251, 500, "B1")
cards += one_form(c, 'INDICATIVO_PASSATO_PROSSIMO',    1, 501, 1000, "B2")

cards += one_form(c, 'INDICATIVO_FUTURO_SEMPLICE',     1, 0, 250, "A2")
cards += one_form(c, 'INDICATIVO_FUTURO_SEMPLICE',     1, 251, 500, "A2")
cards += one_form(c, 'INDICATIVO_FUTURO_SEMPLICE',     1, 501, 1000, "B2")

cards += one_form(c, 'INDICATIVO_IMPERFETTO',          1, 0, 250, "A2")
cards += one_form(c, 'INDICATIVO_IMPERFETTO',          1, 251, 500, "B1")
cards += one_form(c, 'INDICATIVO_IMPERFETTO',          1, 501, 1000, "B2")

cards += one_form(c, 'CONDIZIONALE_PRESENTE',          1, 0, 250, "A2")
cards += one_form(c, 'CONDIZIONALE_PRESENTE',          1, 251, 500, "B1")
cards += one_form(c, 'CONDIZIONALE_PRESENTE',          1, 501, 1000, "B2")

cards += one_form(c, 'CONGIUNTIVO_PRESENTE',           1, 0, 250, "A2")
cards += one_form(c, 'CONGIUNTIVO_PRESENTE',           1, 251, 500, "B1")
cards += one_form(c, 'CONGIUNTIVO_PRESENTE',           1, 501, 1000, "B2")

cards += one_form(c, 'INDICATIVO_TRAPASSATO_PROSSIMO', 1, 0, 500, "B1")
cards += one_form(c, 'INDICATIVO_TRAPASSATO_PROSSIMO', 1, 501, 1000, "B2")

cards += one_form(c, 'CONGIUNTIVO_IMPERFETTO',         1, 0, 500, "B1")
cards += one_form(c, 'CONGIUNTIVO_IMPERFETTO',         1, 501, 1000, "B2")

cards += one_form(c, 'CONGIUNTIVO_PASSATO',            1, 0, 500, "B1")
cards += one_form(c, 'CONGIUNTIVO_PASSATO',            1, 501, 1000, "B2")

cards += one_form(c, 'CONGIUNTIVO_TRAPASSATO',         1, 0, 500, "B1")
cards += one_form(c, 'CONGIUNTIVO_TRAPASSATO',         1, 501, 1000, "B2")

cards += one_form(c, 'CONDIZIONALE_PASSATO',           1, 0, 500, "B1")
cards += one_form(c, 'CONDIZIONALE_PASSATO',           1, 501, 1000, "B2")

cards += one_form(c, 'INDICATIVO_PASSATO_REMOTO',      1, 0, 1000, "B2")
cards += one_form(c, 'INDICATIVO_TRAPASSATO_REMOTO',   1, 0, 1000, "B2")
cards += one_form(c, 'INDICATIVO_FUTURO_ANTERIORE',    1, 0, 1000, "B2")

In [52]:
len(cards)

1350

In [ ]:
import urllib
url = 'https://conjugator.reverso.net/index-italian-1-250.html'

def request_contents(url):
    try:
        contents = urllib.request.urlopen(url)
    except urllib.error.HTTPError:
        return (None)
    return (contents.read().decode(encoding="utf-8", errors="strict"))

In [ ]:
html = request_contents(url)

In [ ]:
from cmd import Cmd

def show_card():
    print('''
    Presente: io canto
    
    futuro: io canterò
    ''')

class MyPrompt(Cmd):
    def __init__(self):
        super().__init__()
        self.counter = 12

    def do_increment(self, input):
        self.counter += 1
        
    def do_0(self, input):
        self.counter = 0
    
    def do_card(self, input):
        show_card()
    
    def do_exit(self, inp):
        print("Bye", self.counter)
        return True
 
    def do_add(self, inp):
        print("Adding '{}'".format(inp))
 
MyPrompt().cmdloop()

In [ ]:
import csv
from collections import defaultdict
import random
import os

url1 = '/Users/davidp/work/verbe-italiani/data/csvs/top236.csv'

forms = [
    'Condizionale Passato',
    'Condizionale Presente',
    'Congiuntivo Imperfetto',
    'Congiuntivo Perfetto',
    'Congiuntivo Presente',
    'Congiuntivo Trapassato',
    'Futuro',
    'Futuro Anteriore',
    'Imperfetto',
    'Passato Remoto',
    'Perfetto',
#    'Presente',
    'Trapassato Prossimo'
]

In [ ]:
random.choice(range(8))

In [ ]:
def write_full_csv(url, cards):
    with open(url, encoding='utf-8', mode='w') as f:
        w = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for card in cards:
            w.writerow(card)

In [ ]:
def read_full_csv(url):
    db = defaultdict(dict)
    with open(url, encoding='utf-8', mode='r') as f:
        r = csv.reader(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for i, row in enumerate(r):
            if i == 0:
                continue
            _, pos, root, form, front, back = row
            if not form in db[root].keys():
                db[root][form] = ['' for i in range(8)]            
            db[root][form][int(pos)] = back
    return(db)

In [ ]:
x = 'Abc'
'-'.join(x.lower().split(' '))

In [ ]:
def one_form(db, form):
    keys = list(db.keys())
    random.shuffle(keys)
    
    cards = []
    imperfetto = []
    for root in db.keys():
        presente = db[root]['Presente']
        i = random.choice(range(8))
        front = '%s --> %s' %(presente[i], form)
        back = db[root][form][i]
        cards += [[front, back]]
    return(cards)
    
def main(url):
    db = read_full_csv(url1)
    for form in forms:
        cards = one_form(db, form)
        base, ext = os.path.splitext(url)
        suffix = '-'.join(form.lower().split(' '))
        output = '%s-%s%s' %(base, suffix, ext)
        write_full_csv(output, cards)

main(url1)